In [3]:
import numpy as np
import networkx as nx
import pandas as pd
import anndata as ad
from anndata import AnnData
import scanpy as sc

In [92]:
adata_exp0 = ad.read_h5ad('C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/spatial/counts_CPc_exp0_BA28.h5ad')

In [76]:
adata_Yao = ad.read_h5ad(
    'C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/sc/Yao_150kcells_subsample_with_annotations_sparse_subset.h5ad')

In [77]:
adata_Yao.obs["celltype"] = adata_Yao.obs["label"]

In [95]:
sc.pp.normalize_total(adata_exp0)
sc.pp.normalize_total(adata_Yao)
# sc.pp.log1p(adata_exp0)
# sc.pp.log1p(adata_Yao)
adata_exp0.layers["lognorm"] = adata_exp0.X
adata_Yao.layers["lognorm"] = adata_Yao.X

In [94]:
def get_knn_mixing_score(adata_st: AnnData, adata_sc: AnnData, obs_key: str = "celltype", k: int = 45,ct_filter_factor: float = 2):
    adata_st.obs["modality"] = "spatial"
    adata_sc.obs["modality"] = "sc"
    adata = ad.concat([adata_st, adata_sc], join='inner')  
    adata.X = adata.X.astype(np.float32)
    adata.obs_names = adata.obs_names.astype(str) + "_" + adata.obs["modality"].astype(str)
    adata.obs = adata.obs.reset_index()

    # Set counts to log norm data
    adata.X = adata.layers["lognorm"]
    
    # Calculate PCA (Note: we could also think about pca per cell type...)
    assert (adata.obsm is None) or ('X_pca' not in adata.obsm), "PCA already exists."
    sc.tl.pca(adata)
    
    # get cell type groups
    sc_cts = set(adata_sc.obs["celltype"].cat.categories)
    st_cts = set(adata_st.obs["celltype"].cat.categories)
    shared_cts = list(sc_cts.intersection(st_cts))         

    # Get ratio per shared cell type
    df = pd.DataFrame(columns=["index","score"])
    for ct in shared_cts:
        enough_cells = (adata.obs.loc[adata.obs[obs_key]==ct,"modality"].value_counts() > (ct_filter_factor * k)).all()     #nochmal: wieso ct_fil?
        if enough_cells:
            a = adata[adata.obs[obs_key]==ct]
            exp_val = (a.obs.loc[a.obs["modality"]=="sc"].shape[0])/a.obs.shape[0]  #sinnvoller EW?
            sc.pp.neighbors(a,n_neighbors=k)
            G = nx.Graph(incoming_graph_data=a.obsp["connectivities"])
            nx.set_node_attributes(G, {i:a.obs["modality"].values[i] for i in range(G.number_of_nodes())}, "modality")   

            ct_df = np.zeros(a.obs.shape[0])
            f = np.vectorize(lambda x: x/exp_val if x>=0 and x<=exp_val else x/(exp_val-1)+1/(1-exp_val))
            i = 0
            for cell in G.nodes():
                ct_df[i] = sum(1 for neighbor in G.neighbors(cell) if G.nodes[cell]["modality"]=="sc")  #number_modality_sc
                ct_df[i] = ct_df[i]/G.degree(cell)      #ratio: number modality sc / total cells
                i += 1 

            df_t = a.obs[a.obs['index'].str.endswith('spatial')]    	    #TODO: früher subsetten
            df_t["index"] = df_t["index"].apply(lambda s: s.rsplit('_',1)[0])
            ct_df = ct_df[a.obs['index'].str.endswith('spatial')]
            
            daf = pd.DataFrame({"index": df_t["index"], "score":f(ct_df)})
            df = pd.concat([df,daf])

    adata_st.obs = pd.merge(adata_st.obs.reset_index(),df,left_on="index",right_on="index",how="left")   
         

In [97]:
adata_exp0.obs

,index,cell_id,ct_majority,ct_majority_cert,celltype,n_counts,n_genes,area,modality,score
0,Cell_0,1855,CA3,0.859629,CA3,862.0,104,7410.0,spatial,0.0
1,Cell_1,21448,L6 CT CTX,1.000000,L6 CT CTX,35.0,20,361.7,spatial,0.0
2,Cell_2,1464,Endo,1.000000,Endo,85.0,28,681.2,spatial,0.0
3,Cell_3,1646,CA3,0.594203,CA3,276.0,71,2597.0,spatial,0.0
4,Cell_4,21882,Oligo,1.000000,Oligo,39.0,16,485.3,spatial,0.0
...,...,...,...,...,...,...,...,...,...,...
23277,Cell_23277,22767,Micro-PVM,1.000000,Micro-PVM,2.0,2,NaN,spatial,0.0
23278,Cell_23278,22837,Endo,1.000000,Endo,2.0,1,NaN,spatial,0.0
23279,Cell_23279,23113,Endo,1.000000,Endo,1.0,1,NaN,spatial,0.0
23280,Cell_23280,23114,CR,1.000000,CR,2.0,2,NaN,spatial,NaN


In [96]:
#test get_modality_ratio_score
get_knn_mixing_score(adata_exp0, adata_Yao)       

c:\Users\mdichgan\AppData\Local\miniconda3\envs\txsim\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
C:\Users\mdichgan\AppData\Local\Temp\ipykernel_7920\2151726017.py:28: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  sc.pp.neighbors(a,n_neighbors=k)
C:\Users\mdichgan\AppData\Local\Temp\ipykernel_7920\2151726017.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t["index"] = df_t["index"].apply(lambda s: s.rspli